In [1]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

In [2]:
from openai_hackathon_wait.tools.arxiv_tool import arxiv_search

In [3]:
PAPER_CONTENT = """
# Abstract

We present PPCEF, a novel method for generating prob-
abilistically plausible counterfactual explanations (CFs). PPCEF ad-
vances beyond existing methods by combining a probabilistic formula-
tion that leverages the data distribution with the optimization of plausi-
bility within a unified framework. Compared to reference approaches,
our method enforces plausibility by directly optimizing the explicit
density function without assuming a particular family of parametrized
distributions. This ensures CFs are not only valid (i.e., achieve class
change) but also align with the underlying data’s probability density.
For that purpose, our approach leverages normalizing flows as power-
ful density estimators to capture the complex high-dimensional data
distribution. Furthermore, we introduce a novel loss function that bal-
ances the trade-off between achieving class change and maintaining
closeness to the original instance while also incorporating a proba-
bilistic plausibility term. PPCEF’s unconstrained formulation allows
for an efficient gradient-based optimization with batch processing,
leading to orders of magnitude faster computation compared to prior
methods. Moreover, the unconstrained formulation of PPCEF allows
for the seamless integration of future constraints tailored to specific
counterfactual properties. Finally, extensive evaluations demonstrate
PPCEF’s superiority in generating high-quality, probabilistically plau-
sible counterfactual explanations in high-dimensional tabular settings.

# Keywords
Counterfactual Explanations, Probabilistic Plausibility, Normalizing Flows, Explainable AI (XAI)
"""

In [4]:
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel
import asyncio

arxiv_agent = Agent(
    name="Arxiv agent",
    instructions="You provide information about the papers that are related to the query. Return between 15 and 25 results by default.",
    tools=[arxiv_search],
    model="gpt-4o-mini",
    output_type=list[str],
)

triage_agent = Agent(
    name="Triage agent",
    instructions="You are a helpful assistant that finds the most relevant papers from Arxiv.",
    handoffs=[arxiv_agent],
    model="gpt-4o-mini",
    output_type=list[str],
)

In [5]:
async def main():
    result = await Runner.run(triage_agent, input=PAPER_CONTENT)
    print(result.final_output)

In [6]:
res = await main()

2025-04-26 02:11:35.359 | INFO     | openai_hackathon_wait.tools.arxiv_tool:arxiv_search:60 - Max results: 20
2025-04-26 02:11:35.360 | INFO     | openai_hackathon_wait.tools.arxiv_tool:arxiv_search:61 - Sort by: relevance
2025-04-26 02:11:35.360 | INFO     | openai_hackathon_wait.tools.arxiv_tool:arxiv_search:73 - Starting arXiv search for: probabilistic counterfactual explanations, normalizing flows, explainable AI
2025-04-26 02:11:38.040 | INFO     | openai_hackathon_wait.api.utils.get_article_keywords:get_article_keywords:42 - Article keywords response: keywords=['probabilistic counterfactual explanations', 'counterfactual reasoning', 'explainable artificial intelligence', 'explainability in AI', 'normalizing flows', 'probabilistic models', 'causal inference', 'machine learning interpretability', 'generative models', 'latent variable models', 'Bayesian inference', 'AI transparency', 'decision-making in AI', 'model interpretability', 'data-driven explanations', 'causal models', 'adv

['Probabilistic Counterfactual Explanations via Normalizing Flows: https://arxiv.org/pdf/2102.05460v2', 'Counterfactual Explanations with Deep Learning Models: https://arxiv.org/pdf/1911.05747', 'On the Stability of Counterfactual Explanations with Respect to Stable Features: https://arxiv.org/pdf/2106.04494', 'Interpretable deep learning through counterfactual explanations: https://arxiv.org/pdf/2008.09103', 'Counterfactual Explanations for Machine Learning: A Survey: https://arxiv.org/pdf/2010.01829', 'Generating Counterfactuals for Feature Attribution based Explanations: https://arxiv.org/pdf/2105.07845', 'Towards Robust Counterfactual Explanations: https://arxiv.org/pdf/2002.07807', 'Counterfactuals for Classification Models: A Survey: https://arxiv.org/pdf/2005.09227', 'Counterfactual Explanations in Machine Learning: Best Practices and Examples: https://arxiv.org/pdf/2108.07223', 'Towards Fairness in Machine Learning: Counterfactual Explanations with Normalizing Flows: https://ar

In [7]:
res